In [1]:
stock = 'SNAP'
days_back = 100

In [2]:
import pandas as pd                              # 表格和数据处理
import matplotlib.pyplot as plt                  # 绘图
import seaborn as sns                            # 更多绘图功能
import pandas_datareader.data as webdata

from itertools import product                    # 一些有用的函数
from tqdm import tqdm_notebook

import datetime                                  # 日期函数

import warnings                                  # 勿扰模式
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
now = datetime.date.today()
start = now - datetime.timedelta(days=days_back)
# df = data.DataReader(stock, "iex", start=start, end=end)
df = webdata.get_data_yahoo(stock, start=start, end=now)['Adj Close']

In [4]:
df

Date
2018-07-23    13.39
2018-07-24    13.14
2018-07-25    13.39
2018-07-26    13.37
2018-07-27    12.83
2018-07-30    12.27
2018-07-31    12.50
2018-08-01    12.38
2018-08-02    12.70
2018-08-03    12.73
2018-08-06    13.05
2018-08-07    13.12
2018-08-08    12.23
2018-08-09    12.31
2018-08-10    12.31
2018-08-13    12.57
2018-08-14    12.34
2018-08-15    12.18
2018-08-16    12.17
2018-08-17    11.89
2018-08-20    11.73
2018-08-21    11.44
2018-08-22    11.44
2018-08-23    11.40
2018-08-24    11.63
2018-08-27    11.81
2018-08-28    11.29
2018-08-29    11.17
2018-08-30    11.08
2018-08-31    10.90
              ...  
2018-09-18     9.13
2018-09-19     9.16
2018-09-20     9.21
2018-09-21     9.14
2018-09-24     8.95
2018-09-25     9.05
2018-09-26     8.99
2018-09-27     8.80
2018-09-28     8.48
2018-10-01     8.36
2018-10-02     8.18
2018-10-03     8.23
2018-10-04     7.80
2018-10-05     7.77
2018-10-08     7.48
2018-10-09     7.00
2018-10-10     6.59
2018-10-11     6.81
2018-10-12     

In [5]:
# 需要预测的5天日期
d = [None] * 5
j = 0

for i in range(0,7):
    if (now + datetime.timedelta(i)).weekday() < 5:
        d[j] = str(now + datetime.timedelta(i))
        j = j + 1

In [6]:
#时序数据上的线性模型
data = pd.DataFrame(df.copy().astype(float))
data.columns = ["y"]

df = pd.DataFrame({'y':[0,0,0,0,0]},
                   index = d)
data = data.append(df)

for i in range(6, 25):
    data["lag_{}".format(i)] = data.y.shift(i)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

y = data.dropna().y # 删除n/a的数据
X = data.dropna().drop(['y'], axis=1)

# 除去需要预测的5天
train_index = int(len(X)) - 5
X_train = X.iloc[:train_index]
y_train = y.iloc[:train_index]

# 机器学习
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X.tail(5))
precentage = prediction.mean()
print(prediction.mean())
print(prediction)
# plt.figure(figsize=(15, 7))
# plt.plot(prediction, "g", label="prediction", linewidth=2.0)


5.433292385142205
[5.60453004 5.4480085  5.25425888 5.58162389 5.2780406 ]


In [7]:
#储存数据到excel
from openpyxl import load_workbook
df = pd.DataFrame(prediction)
df = df.values.T.tolist()
stock_name = [str(stock)]
wb = load_workbook('/Users/BartSu/Desktop/StockRS_Result.xlsx')
ws = wb.worksheets[0]

ws.append(stock_name)

# Append 2 new Rows - Columns A - D
for row_data in df:
    # Append Row Values
    ws.append(row_data)

wb.save("/Users/BartSu/Desktop/StockRS_Result.xlsx")